给你一个整数数组 nums 和一个整数 target 。

向数组中的每个整数前添加 '+' 或 '-' ，然后串联起所有整数，可以构造一个 表达式 ：

例如，nums = [2, 1] ，可以在 2 之前添加 '+' ，在 1 之前添加 '-' ，然后串联起来得到表达式 "+2-1" 。
返回可以通过上述方法构造的、运算结果等于 target 的不同 表达式 的数目。

 

示例 1：

输入：nums = [1,1,1,1,1], target = 3
输出：5
解释：一共有 5 种方法让最终目标和为 3 。
-1 + 1 + 1 + 1 + 1 = 3
+1 - 1 + 1 + 1 + 1 = 3
+1 + 1 - 1 + 1 + 1 = 3
+1 + 1 + 1 - 1 + 1 = 3
+1 + 1 + 1 + 1 - 1 = 3

来源：力扣（LeetCode）
链接：https://leetcode-cn.com/problems/target-sum
著作权归领扣网络所有。商业转载请联系官方授权，非商业转载请注明出处。

In [19]:
# dp[i][j]用前i个数凑j的方法数 [0,len(nums)] [-s,s]->[0,2s]
# dp[i][j]=dp[i-1][j-curNum]+dp[i-1][j+curNum]
# dp[0][n]=0 dp[0][s]=1
def f(nums, target):
    s = sum(nums)
    if abs(target) > s:
        return 0
    dp = [[-1] * (2 * s + 1) for _ in range(len(nums) + 1)] # 向右偏移s
    # basecase 0个数凑不出 除非0个数凑出0(偏移后s) 无为而治一个方法
    for j in range(2 * s + 1):
        dp[0][j] = 0
    dp[0][s] = 1
    for i in range(1, len(nums) + 1):
        curNum = nums[i - 1]
        for j in range(2 * s + 1):
            tmp1 = dp[i - 1][j - curNum] if j - curNum >= 0 else 0 # 加当前数 需要加之前大于最小值-s 偏移后0 否则会出界
            tmp2 = dp[i - 1][j + curNum] if j + curNum <= 2 * s else 0 # 减去当前数 需要减之前小于最大值s 偏移后 2s 否则会出界
            dp[i][j] = tmp1 + tmp2
    return dp[len(nums)][target + s]


f([1, 1, 1, 1, 1], 3)
f([1], 2)


0

In [1]:
from functools import lru_cache


def f(nums, target):

    @lru_cache(None)
    def dfs(i, s):
        if i > len(nums) - 1:
            if s == target:
                return 1
            else:
                return 0
        return dfs(i + 1, s + nums[i]) + dfs(i + 1, s - nums[i])

    return dfs(0, 0)


f([1, 1, 1, 1, 1], 3)


5

In [18]:
from functools import lru_cache


def f(nums, target):

    mem = {}

    def dfs(i, s):
        if (i, s) in mem:
            return mem[(i, s)]
        if i > len(nums) - 1:
            if s == target:
                return 1
            else:
                return 0
        mem[(i, s)] = dfs(i + 1, s + nums[i]) + dfs(i + 1, s - nums[i])
        return mem[(i, s)]

    return dfs(0, 0)


f([1, 1, 1, 1, 1], 3)


5

In [28]:
# S有正子集S1 负子集S2
# 所以有sum(S)=sum(S1)+sum(S2)
# sum(S1)-sum(S2)=target
# sum(S1)-(sum(S)-sum(S1))=target
# sum(S1)=(sum(S)+target)/2
# -> 在nums中找到和为(sum(S)+target)/2 = T 的组合个数 01背包
# dp[i][j] 在前i个数中找到和为j的组合数 [0,len(nums)] [0,T]
# dp[0][j]=0 dp[0][0]=1
def f(nums, target):
    s = sum(nums)
    if abs(target) > s or (s + target) % 2 == 1:
        return 0
    T = (s + target) // 2
    dp = [[-1] * (T + 1) for _ in range(len(nums) + 1)]
    for j in range(T + 1):
        dp[0][j] = 0
    dp[0][0] = 1
    for i in range(1, len(nums) + 1):
        curNum = nums[i - 1]
        for j in range(T + 1):
            if curNum > j:
                dp[i][j] = dp[i - 1][j]
            else:
                dp[i][j] = dp[i - 1][j] + dp[i - 1][j - curNum]
    return dp[-1][-1]


f([1, 1, 1, 1, 1], 3)
f([0, 0, 0, 0, 0, 0, 0, 0, 1], 1)


0

In [ ]:
# S有正子集S1 负子集S2
# 所以有sum(S)=sum(S1)+sum(S2)
# sum(S1)-sum(S2)=target
# sum(S1)-(sum(S)-sum(S1))=target
# sum(S1)=(sum(S)+target)/2
# -> 在nums中找到和为(sum(S)+target)/2 = T 的组合个数 01背包
# dp[i][j] 在前i个数中找到和为j的组合数 [0,len(nums)] [0,T]
# dp[0][j]=0 dp[0][0]=1
def f(nums, target):
    Sum = sum(nums)
    if abs(target) > Sum or (Sum + target) % 2 == 1:
        return 0
    t = (Sum + target) // 2
    # dp[i] 凑t的方法数
    dp = [0] * (t + 1)
    dp[0] = 1
    for c in nums:
        for i in range(t, c - 1, -1):
            dp[i] += dp[i - c]
    return dp[-1]